In [ ]:
import pandas as pd
finance_df = pd.read_excel('재무데이터.xlsx')
finance_df.head(3)

In [ ]:
# pd.options.display.float_format = '{: .5f}'.format
pd.reset_option('display.float_format')

In [ ]:
finance_df.rename(columns={'사업자번호': 'BIZ_NO'}, inplace=True)

In [ ]:
fin_df = finance_df

In [ ]:
fin_df['결산년월']= fin_df['결산년월'].astype('str')
fin_df['결산년월'] = pd.to_datetime(fin_df['결산년월'], format='%Y-%m-%d', errors='raise')

In [ ]:
fin_df['결산년월']

In [ ]:
fin_df['year'] = fin_df.결산년월.apply(lambda x: x.year)
fin_df['year']

In [ ]:
fin = fin_df[['BIZ_NO','year','유보액/총자산(%)', '기업순이익률(%)','매출액순이익률(%)','차입금의존도(%)','부채총계대 매출액(%)']]

In [ ]:
fin.head()

In [ ]:
active_df = pd.read_excel('액티브 중소법인 재무보유.xlsx')
close_df = pd.read_excel('휴폐업 중소법인 재무보유.xlsx')

In [ ]:
active_df['open'] = 0
close_df['open'] = 1

In [ ]:
close_df.describe()


In [ ]:
all_data = active_df.append(close_df,ignore_index=True)
all_data

In [ ]:
all_data.describe()

In [ ]:
all_data = all_data[['BIZ_NO', 'BZ_TYP','VENT_YN','MDSCO_PRTC_YN','ESTB_DATE','EMP_CNT','open']]
all_data

In [ ]:
all_data = pd.merge(all_data, fin, how='outer', on='BIZ_NO')

In [ ]:
all_data

In [ ]:
all_data.isna().sum()

In [ ]:
all_data['ESTB_DATE'].dropna(axis=0)

In [ ]:
mask = all_data['ESTB_DATE'].isin(['-9223372036854775808'])
all_data = all_data[~mask]

In [ ]:
all_data['ESTB_DATE']= all_data['ESTB_DATE'].astype('str')
all_data['ESTB_DATE'] = pd.to_datetime(all_data['ESTB_DATE'],format='%Y%m%d', errors='raise')

In [ ]:
all_data['ESTB_DATE'].info()

In [ ]:
all_data['ESTB_year'] =all_data.ESTB_DATE.apply(lambda x: x.year)
all_data.drop(['ESTB_DATE'], axis=1, inplace=True)

In [ ]:
all_data['ESTB_year']
all_data

In [ ]:
all_data.drop(['BIZ_NO'], axis=1, inplace=True)
all_data

In [ ]:
all_data['ESTB_year']= all_data['ESTB_year'].astype('str')
all_data['year'] = all_data['year'].astype('str')

In [ ]:
all_data.info()

In [ ]:
all_data.isna().sum()

In [ ]:
train_isna_sum = all_data.isna().sum()
train_isna_sum[train_isna_sum != 0].index
na_columns = train_isna_sum[train_isna_sum != 0].index

def fill_bicycle_na(df, column):
    df[column] = df[column].fillna(value = df[column].mean())

for col in na_columns:
    fill_bicycle_na(all_data, col)
    print(col, '결측값 대체 완료')

In [ ]:
all_data_ohe = pd.get_dummies(all_data, columns=['BZ_TYP', 'VENT_YN', 'year','MDSCO_PRTC_YN','ESTB_year'])

In [ ]:
all_data_ohe.drop(['ESTB_year_nan','year_nan'], axis=1, inplace=True)

In [ ]:
all_data_ohe

In [ ]:
from tkinter.font import families
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='AppleGothic')
all_data_ohe.boxplot(column=['유보액/총자산(%)','기업순이익률(%)','매출액순이익률(%)','차입금의존도(%)','부채총계대 매출액(%)'])

In [ ]:
all_data_ohe[['유보액/총자산(%)','기업순이익률(%)','매출액순이익률(%)','차입금의존도(%)','부채총계대 매출액(%)']].info()

## 이상치 제거

In [ ]:
q3 = all_data_ohe.quantile(0.75) # df['국어'].quantile(0.75) 처럼 특정 열만 적용 가능
q1 = all_data_ohe.quantile(0.25)

iqr = q3 - q1

In [ ]:
# '국어' 열에 대하여 이상치 여부를 판별해주는 함수
columns = ['유보액/총자산(%)','기업순이익률(%)','매출액순이익률(%)','차입금의존도(%)','부채총계대 매출액(%)']

def is_kor_outlier(df):
    kor_score = df['차입금의존도(%)']
    if kor_score > q3['차입금의존도(%)'] + 1.5 * iqr['차입금의존도(%)'] or kor_score < q1['차입금의존도(%)'] - 1.5 * iqr['차입금의존도(%)']:
        return True
    else:
        return False

# apply 함수를 통하여 각 값의 이상치 여부를 찾고 새로운 열에 결과 저장
all_data_ohe['차입금의존도(%)_이상치여부'] = all_data_ohe.apply(is_kor_outlier, axis = 1) # axis = 1 지정 필수

all_data_ohe

In [ ]:
df_trim = all_data_ohe.loc[all_data_ohe['유보액/총자산(%)_이상치여부'] == False]
df_trim = df_trim.loc[df_trim['기업순이익률(%)_이상치여부'] == False]

df_trim = df_trim.loc[df_trim['매출액순이익률(%)_이상치여부'] == False]
df_trim = df_trim.loc[df_trim['차입금의존도(%)_이상치여부'] == False]
df_trim = df_trim.loc[df_trim['부채총계대 매출액(%)_이상치여부'] == False]

df_trim.drop(['유보액/총자산(%)_이상치여부','기업순이익률(%)_이상치여부','매출액순이익률(%)_이상치여부','차입금의존도(%)_이상치여부','부채총계대 매출액(%)_이상치여부'], axis=1, inplace=True)

all_data_ohe = df_trim

In [ ]:
all_data_ohe

In [ ]:
from tkinter.font import families
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='AppleGothic')
all_data_ohe.boxplot(column=['유보액/총자산(%)','기업순이익률(%)','매출액순이익률(%)','차입금의존도(%)','부채총계대 매출액(%)'])

## 앙상블 학습

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    f1 = f1_score(y_test, pred)
    
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도:{1: .4f}, 재현율:{2: .4f}, f1점수: {3: .4f}'.format(accuracy, precision, recall, f1))
    print('AUC점수:{0: .4f}'.format(roc_auc))

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

lr_clf = LogisticRegression(solver='liblinear')
knn_clf = KNeighborsClassifier(n_neighbors=8)

vo_clf = VotingClassifier(estimators=[('LR', lr_clf),('KNN', knn_clf)], voting='soft')
X_target = all_data_ohe['open']
X_features = all_data_ohe.drop(['open'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X_features, X_target, test_size=0.3, random_state=0)

vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)
pred_proba = vo_clf.predict_proba(X_test)[:,1]
print('Voting 분류기 정확도:{0: .4f}'.format(accuracy_score(y_test, pred)))
get_clf_eval(y_test, pred, pred_proba)

In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors=8)
knn_clf.fit(X_train, y_train)
knn_pred = knn_clf.predict(X_test)
knn_pred_proba = knn_clf.predict_proba(X_test)[:,1]
print('Voting 분류기 정확도:{0: .4f}'.format(accuracy_score(y_test, knn_pred)))
get_clf_eval(y_test, knn_pred, knn_pred_proba)

## 책따라(신용카드 사기 검출)

In [ ]:
y_target = all_data_ohe['open']
X_features = all_data_ohe.drop(['open'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.3, random_state=0, stratify=X_target)

print('학습 데이터 레이블 값 비율')
print(y_train.value_counts()/y_train.count())
print('테스트 데이터 비율')
print(y_test.value_counts()/y_test.count())

In [ ]:
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:, 1]
    get_clf_eval(tgt_test, pred)

In [ ]:
all_data_ohe.dropna(axis=0)

In [ ]:
all_data_ohe.describe()

In [ ]:
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
lgbm_clf.fit(X_train, y_train)
lgbm_pred = lgbm_clf.predict(X_test)
lgbm_pred_proba = lgbm_clf.predict_proba(X_test)[:,1]
get_clf_eval(y_test, lgbm_pred, lgbm_pred_proba)

# get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
# plt.xticks(range(0,30000, 1000), rotation=60)
sns.histplot(all_data_ohe['매출액순이익률(%)'], bins=100, kde=True)
plt.show()